In [ ]:
import json
import os
from glob import glob

import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [ ]:
y = pd.read_csv('train_anwers.csv', sep='\t')
print(y.dtypes)
y.head()

name      int64
target     bool
dtype: object


,name,target
0,29970,True
1,18654,False
2,29133,False
3,20927,False
4,5526,False


In [ ]:
raw_data = {}
for name in y['name']:
    with open(f'train_data/{name}.json', 'r', encoding='utf-8') as inp:
        raw_data[name] = json.load(inp) 

In [ ]:
def extract_simple_features(data) -> np.ndarray:
    res = np.zeros(10, dtype=int)
    players = data['players']
    for i, p in enumerate(players):
        res[i] = p['kda']
    return res

In [ ]:
X = np.zeros((len(raw_data), 10), dtype=int)
for i, data in enumerate(raw_data.values()):
    X[i] = extract_simple_features(data)

In [ ]:
clf = LogisticRegression()
cross_val_score(clf, X, y['target'],scoring='f1')


array([0.86507937, 0.8380829 , 0.86742857, 0.82330097, 0.85631518])

In [ ]:
clf = LogisticRegression()
clf.fit(X, y['target'])

LogisticRegression()

In [ ]:
raw_data_to_predict = {}
for file in glob('public_data/*'):
    key = int(os.path.basename(file).split('.')[0])
    with open(file, 'r', encoding='utf-8') as inp:
        raw_data_to_predict[key] = json.load(inp) 

In [ ]:
X_to_predict = np.zeros((len(raw_data_to_predict), 10), dtype=int)
for i, data in enumerate(raw_data_to_predict.values()):
    X_to_predict[i] = extract_simple_features(data)

In [ ]:
answer = pd.DataFrame()
answer['name'] = raw_data_to_predict.keys()
answer['target'] = clf.predict(X_to_predict)

In [ ]:
answer.to_csv('simple_answer.csv', sep='\t', index=False)